## Importing libraries

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import numpy as np

## Loading Data

In [3]:
df = pd.read_csv('base_k9_c_pca_v1.csv')
df = df.set_index('DNI')

cols_to_add = ['DNI', 'ING_ESTIMADO_1', 'ING_ESTIMADO_2', 'ING_ESTIMADO_3', 'ING_ESTIMADO_4', 'ING_ESTIMADO_5',
               'Promedio_ing', 'Standard_deviation', 'CONSUMO_GENUINO', 'Consumo_total', 'COMPRA_RANK1',
               'COMPRA_RANK2', 'COMPRA_RANK3', 'COMPRA_RANK4', 'COMPRA_RANK5', 'CATEGORIA_RANK1',
               'CATEGORIA_RANK2', 'CATEGORIA_RANK3', 'CATEGORIA_RANK4', 'CATEGORIA_RANK5', 'q_reclamos_nol_3m',
               'q_reclamos_call_3m', 'q_reclamos_fisico_3m', 'utiliz_cred', 'Cupones_total', 'Cupones_promo',
               'SEXO', 'LOCALIDAD', 'PROVINCIA', 'BARRIO']

df_orig = pd.read_csv('data_20200827_080926.csv', usecols=cols_to_add)
df_orig = df_orig.set_index('DNI')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (34,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.



In [4]:
df = pd.merge(df, df_orig, how='left', left_index=True, right_index=True)

df_full_orig = df.copy()

In [5]:
fig = make_subplots(rows=1, cols=1)


fig.add_trace(go.Pie(labels=df['cluster_label'].value_counts().index, 
              values=list(df['cluster_label'].value_counts()),
              title = 'CLUSTERS',
              scalegroup='one', 
              hole=0.6
                    
                    ))

fig.update_layout(
                  autosize=False,
                  width=600,
                  height=600
)
fig.show()
fig.write_html(f'clusters_distribution.html')

In [9]:
df_full_orig.to_csv('dataset_full_with_cluster_labels.csv')

In [ ]:
categorical_columns = [ 'CATEGORIA_RANK1',
               'CATEGORIA_RANK2', 'CATEGORIA_RANK3', 'CATEGORIA_RANK4', 'CATEGORIA_RANK5', 
               'SEXO', 'LOCALIDAD', 'PROVINCIA', 'BARRIO']

numerical_columns = [c for c in df_full_orig.columns if not c in categorical_columns and not 'cluster_label' in c 
                     and not 'dist_to_centroid' in c]

## RF Models Training

In [ ]:
# Encoding categorical variables

from sklearn import preprocessing

for c in categorical_columns:
    
    # Before, I handle missing values
    print(f'Processing column {c}...')
    df[c] = df[c].fillna('NaN')
    
    le = preprocessing.LabelEncoder()
    df[c] = le.fit_transform(df[c])

In [ ]:
# Function to obtain importance of features

def get_lgbm_varimp(model, train_columns, max_vars=50):
    
    cv_varimp_df = pd.DataFrame([train_columns, model.feature_importances_]).T
    
    cv_varimp_df.columns = ['feature_name', 'varimp']
    
    cv_varimp_df.sort_values(by='varimp', ascending=False, inplace=True)
    
    cv_varimp_df = cv_varimp_df.iloc[0 : max_vars]
    
    return cv_varimp_df

In [ ]:
print('Pass')

In [ ]:
#X_full_wo_transf = X_full_wo_transf.rename(columns=columns_to_ascii)
from lightgbm import LGBMClassifier

model = LGBMClassifier(learning_rate=0.1, n_estimators=300, boosting_type='dart', importance_type='gain')

In [ ]:
aux_t_train = df.drop('dist_to_centroid', axis=1)

In [ ]:
#aux_t_train = aux_t_train.drop([c for c in cols_to_add if not 'DNI' in c], axis=1)

In [ ]:
from sklearn.metrics import classification_report

models_dict={}

#for i in range(n_cluster_select)
#    models = {}
for i in df['cluster_label'].value_counts().index:
     
    target = [1 if c == i else 0 for c in df['cluster_label']]

    model.fit(aux_t_train.drop('cluster_label', axis=1), target, eval_metric='auc',
              verbose=100, categorical_feature=[c for c in categorical_columns if 'SEXO' not in c])
    
    y_pred = model.predict(aux_t_train.drop('cluster_label', axis=1))
    classif = classification_report(target, y_pred)
    important_features = get_lgbm_varimp(model, aux_t_train.drop('cluster_label', axis=1).columns, max_vars=50)
    
    model_properties = {
        'trained_model': model,
        'classification_report': classif,
        'features_importance': important_features
    }
    
    models_dict.update({i: model_properties})

In [ ]:
import pickle

file_to_write = open("rf_models_dict.pickle", "wb")

pickle.dump(models_dict, file_to_write)

file_to_write.close()

In [ ]:
# Loading models_dict

import pickle

file_to_read = open("rf_models_dict.pickle", "rb")
models_dict = pickle.load(file_to_read)
file_to_read.close()

In [ ]:
sorted(models_dict.keys())

In [ ]:
from openpyxl import load_workbook

    
writer = pd.ExcelWriter('features_importances_by_cluster.xlsx')

# Importancia de features por cluster
for i in sorted(models_dict.keys()):
    print(f'Cluster {i}:')
    
    #print(models_dict[i]['features_importance'])
    #models_dict[i]['features_importance'].to_excel(r'features_importance.xlsx', sheet_name=f'Cluster {i}', index = False)
    #print()
    # write out the new sheet
    models_dict[i]['features_importance'].to_excel(writer,f'Cluster_{i}', index = False)
writer.save()

## Generation of tables with statistical info for each cluster

Se distribuyen en tablas para variables categóricas y numéricas por separado.

In [ ]:
writer_num = pd.ExcelWriter('statistical_info_numerical_features.xlsx')
writer_cat = pd.ExcelWriter('statistical_info_categorical_features.xlsx')


for cluster in sorted(df_full_orig['cluster_label'].unique()):
    num = df_full_orig[df_full_orig['cluster_label']== \
                 cluster][numerical_columns].describe()
    
    num.to_excel(writer_num,f'Cluster_{cluster}', index = True)
    
    cat = df_full_orig[df_full_orig['cluster_label']== \
                 cluster][categorical_columns].describe()
    
    cat.to_excel(writer_cat,f'Cluster_{cluster}', index = True)

writer_num.save()
writer_cat.save()

## Graphical Analysis

In [ ]:

def quantile_target_distribution_relative(data, column, nq=5, target='TARGET'):
    X = data[[column,target]].copy()
    if X[column].nunique() > nq:
        try:
            X['INTERVAL'] = pd.qcut( X[column], np.linspace(0,1,nq+1) ).map(str)
        except:
            X['INTERVAL'] = pd.cut( X[column], np.unique(np.quantile(X[column],np.linspace(0,1,nq+1),interpolation='lower')), include_lowest=True ).map(str)
       # Q = sorted(list(X.INTERVAL.unique()))
#         X = X.groupby('INTERVAL')[target].value_counts().rename('COUNTS').reset_index()
#     #     X.COUNTS = X.COUNTS*(2*X.TARGET-1)
#         X[target] = X[target].astype(str)
    else:
        X['INTERVAL'] = X[column]
#        if column in ENCODERS:
#            X['INTERVAL'] = ENCODERS[column].inverse_transform(X[column])
    X = X.groupby('INTERVAL')[target].value_counts(normalize=True).rename('PROPORTION').reset_index()
    X[target] = X[target].astype(str)
    X = X.sort_values(['INTERVAL',target])
#     print(X[target])
    return X

def quantile_target_distribution_plot_relative(data, column, nq=5, target='TARGET'):
    fig = px.bar(quantile_target_distribution_relative(data,column,nq,target),
                x='INTERVAL', y='PROPORTION', color=target, barmode='relative',
                height=400)
    fig.update_layout(title_text=f'{column}')
    fig.update_yaxes(range=[0,1])
    return fig


In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
for i, sub_num_cols in enumerate(chunks(numerical_columns, 10)):
    try:
        df_aux=pd.DataFrame()

        for col in sub_num_cols:

            a = quantile_target_distribution_relative(df_full_orig,  col, 10, target='cluster_label')
            a['col'] = col
            df_aux = pd.concat([df_aux, a])




        fig = px.bar(df_aux,
                        x='INTERVAL', y='PROPORTION', color='cluster_label', barmode='relative',
                        height=10000, facet_row="col")
        fig.update_layout(title_text='Numerical Features')
        fig.update_xaxes(matches=None)
        fig.update_yaxes(range=[0,1])
    
        #figure.update_layout(height=600, width=800, title_text="Numerical Features")
        #fig.show()
        fig.write_html(f'numerical_features_graphs_{i}.html')
    except:
        print(sub_num_cols)



## Ahora analizo las variables categóricas

In [ ]:
df_full_orig[['CATEGORIA_RANK1','cluster_label']].dropna()

In [ ]:
for i, sub_cat_cols in enumerate(chunks(categorical_columns, 10)):
    try:
        df_aux=pd.DataFrame()

        for col in sub_cat_cols:

            
            X = df_full_orig[[col,'cluster_label']].dropna().copy()
            X['INTERVAL'] = X[col]
            #        if column in ENCODERS:
            #            X['INTERVAL'] = ENCODERS[column].inverse_transform(X[column])
            X = X.groupby('INTERVAL')['cluster_label'].value_counts(normalize=True).rename('PROPORTION').reset_index()
            X['cluster_label'] = X['cluster_label'].astype(str)
            X = X.sort_values(['INTERVAL','cluster_label'])
            
            #a = quantile_target_distribution_relative(df_full_orig,  col, 10, target='cluster_label')
            X['col'] = col
            df_aux = pd.concat([df_aux, a])


        fig = px.bar(df_aux,
                        x='INTERVAL', y='PROPORTION', color='cluster_label', barmode='relative',
                        height=10000, facet_row="col")
        fig.update_layout(title_text='Categorical Features')
        fig.update_xaxes(matches=None)
        fig.update_yaxes(range=[0,1])
    
        #figure.update_layout(height=600, width=800, title_text="Numerical Features")
        #fig.show()
        fig.write_html(f'categorical_features_graphs_{i}.html')
    except:
        print(sub_cat_cols)